
### Librerías

In [57]:
import sys
import pandas as pd
import GetOldTweets3 as got
import sys
import matplotlib.pyplot as plt
from googletrans import Translator
import socket as sk
from datetime import *
import tweepy
import re
import string
from textblob import TextBlob
import nltk

### Configuración de parámetros 

In [58]:
pd.options.display.max_columns = 5000
pd.options.display.max_rows = 5000
sk.setdefaulttimeout(600)

#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('max_colwidth',-1)

### Ejecución de clases y métodos del proyecto

In [71]:
%run Metodos_Propios.ipynb
%run Gestion_Ficheros.ipynb
%run GetOldTweets.ipynb
%run NLP.ipynb
%run Extract_Twitter_v2.ipynb

# Método Principal - TFM Data Science

In [73]:
if __name__ == '__main__':
    
    #Control horario programa
    fecha_inicio = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print('Comienzo del programa: ',fecha_inicio)
    
    # Definición de variables
    fichero = 'CSVs/info_tweets_el_corte_ingles_202003.csv'
    pattern = 'CSVs/info_tweets_el_cor*.csv'
    fichero_user = 'CSVs/following_info_el_corte_ingles.csv'
    fichero_tweets = 'CdM/Tweets_Info.csv'
    fichero_fact = 'CdM/Tweets_Fact_Info.csv'
    following_ids = []  
    calendario = calendar()
    df_tweets, df_tweets_final, df_following, df_fact_following = creacion_DataFrame()
    positivo, negativo, neutral, sentiment = creacion_Variables()
    
    #Flujo de la información
    online = 0
    all_tweets = 0
    online_user = 0
    
    #Definición POO
    file = File_Manage()
    file.initValues(fichero, pattern)
    gotTweets = GoT(calendario)
    nlp = NLP()

    
    #Acceso a Twitter
    api = acceso_Twitter() 

    
    #Obtención por GetOldTweets
    if online == 1:
        print('Opción Online')
        print(gotTweets.getCalendar())
        df_tweets = gotTweets.get_Tweets(gotTweets.getCalendar())
    #Lectura de ficheros para ir más rápido ubicados en CSVs
    elif all_tweets == 1:
        print('Opción Offline Tweets')
        df_tweets = file.read_all_Tweets(file.getPattern()) 
    else: 
        print('Opción Un fichero: ', file.getFile())
        df_tweets = file.read_Tweets(file.getFile())
    
    df_tweets['valor'] = -10
    df_tweets['fecha'] = df_tweets['date'].astype(str).str[:10]
    df_tweets['fecha'] = df_tweets['fecha'].str.replace('-','/')
    df_tweets['idioma'] = 'NA' 
    
    df_tweets['Tweet_punct'] = df_tweets['text'].str.lower().apply(lambda text: nlp.eliminar_signos(text))
    df_tweets['Tweet_token'] = df_tweets['Tweet_punct'].apply(lambda text: nlp.tokenization(text))
    df_tweets['Tweet_novacias'] = df_tweets['Tweet_token'].apply(lambda text: nlp.quitar_palabras_vacias(text))


    #Pasando el algoritmo de NLP para obtener el análisis de sentimiento
    df_tweets_final = nlp.analisis_2(df_tweets)

    #Generación de CSVs para el CdM
    file.setFile(fichero_tweets)
    #file.writeCSV(df_tweets_final,file.getFile())
    
    #Para el caso de los usuarios followings, se crea un minimodelo de datos
    if online_user == 1:
        following_ids = get_following_ids(api)
        following_limit=[following_ids[n:n+99] for n in range(0, len(following_ids), 100)]
        df_following = pd.DataFrame(get_info_following(api,following_limit))
        file.setFile(fichero_user)
        file.writeCSV(df_following,file.getFile())
    else:
        file.setFile(fichero_user)
        df_following = file.readCSV(df_following,file.getFile())
     
    df_tweets.columns = ['id_text','created_at_tweet','name_account','text','geo','valor','fecha','idioma','Tweet_punct','Tweet_token','Tweet_novacias']
    df_following.columns = ['id_account','name_account','screen_name_account','created_at_account','location_account','followers_account','friends_account','description_account']
    df_fact_following = pd.merge(df_tweets, df_following,on=['name_account'],how='left')
    df_fact_following = df_fact_following[df_fact_following['id_account'].notnull()]
        
    
    file.setFile(fichero_fact)    
    #file.writeCSV(df_fact_following,file.getFile())
    
    fecha_fin = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print('Comienzo Finalizacion: ',fecha_fin)

Comienzo del programa:  08/09/2020, 19:35:42
Opción Un fichero:  CSVs/info_tweets_el_corte_ingles_202003.csv


NameError: name 'tokenization' is not defined

In [65]:
df_tweets_final.head(100)

,id_text,created_at_tweet,name_account,text,geo,valor,fecha,idioma,text_noSignos,text_token,text_novacias
0,1244775324367323142,2020-03-30 23:55:38+00:00,conchymor,"En serio empresas como el corte inglés, inditex, producción automovilística ...etc, no pueden pagar un mes a sus empleados?",NaN,0,2020/03/30,NA,en serio empresas como el corte inglés inditex producción automovilística etc no pueden pagar un mes a sus empleados,"[en, serio, empresas, como, el, corte, inglés, inditex, producción, automovilística, etc, no, pueden, pagar, un, mes, a, sus, empleados]","[serio, empresas, corte, inglés, inditex, producción, automovilística, etc, pueden, pagar, mes, empleados]"
1,1244773416835715072,2020-03-30 23:48:04+00:00,Nerea_Blake,Al día siguiente fui a El Corte Inglés y me compré el Jersey verde de Ana junto a los pantalones negros y la sudadera blanca y roja de Miriam,NaN,0,2020/03/30,NA,al día siguiente fui a el corte inglés y me compré el jersey verde de ana junto a los pantalones negros y la sudadera blanca y roja de miriam,"[al, día, siguiente, fui, a, el, corte, inglés, y, me, compré, el, jersey, verde, de, ana, junto, a, los, pantalones, negros, y, la, sudadera, blanca, y, roja, de, miriam, ]","[día, siguiente, corte, inglés, compré, jersey, verde, ana, junto, pantalones, negros, sudadera, blanca, roja, miriam, ]"
2,1244772598589816836,2020-03-30 23:44:48+00:00,sarureview,"Pero, a ver, una cosa es una expresión propia del país para referirse a ubñn objeto/alimento/cosa y otra es una cadena de supermercados que es un meme como el Corte Inglés",NaN,0,2020/03/30,NA,pero a ver una cosa es una expresión propia del país para referirse a ubñn objetoalimentocosa y otra es una cadena de supermercados que es un meme como el corte inglés,"[pero, a, ver, una, cosa, es, una, expresión, propia, del, país, para, referirse, a, ubñn, objetoalimentocosa, y, otra, es, una, cadena, de, supermercados, que, es, un, meme, como, el, corte, inglés, ]","[ver, cosa, expresión, propia, país, referirse, ubñn, objetoalimentocosa, cadena, supermercados, meme, corte, inglés, ]"
3,1244770046913712128,2020-03-30 23:34:40+00:00,Julicanales,"El confinamiento ha retratado la penosa distribución online de las grandes empresas de supermercados. Mercadona, Carrefour, El Corte Inglés, Consum... no funciona ninguna.",NaN,0,2020/03/30,NA,el confinamiento ha retratado la penosa distribución online de las grandes empresas de supermercados mercadona carrefour el corte inglés consum no funciona ninguna,"[el, confinamiento, ha, retratado, la, penosa, distribución, online, de, las, grandes, empresas, de, supermercados, mercadona, carrefour, el, corte, inglés, consum, no, funciona, ninguna]","[confinamiento, retratado, penosa, distribución, online, grandes, empresas, supermercados, mercadona, carrefour, corte, inglés, consum, funciona, ninguna]"
4,1244768892146388992,2020-03-30 23:30:05+00:00,daniserna1983,Es que para ella siempre será mejor una donación de Amancio o de El Corte Inglés,NaN,0,2020/03/30,NA,es que para ella siempre será mejor una donación de amancio o de el corte inglés,"[es, que, para, ella, siempre, será, mejor, una, donación, de, amancio, o, de, el, corte, inglés]","[siempre, mejor, donación, amancio, corte, inglés]"
5,1244764809666662402,2020-03-30 23:13:51+00:00,fdkilos,"DIGO LO MISMO yo m iba a comprar melo pq en el corte ingles sale a 4,99 PERO LA MIERDA DEL CORONA PUES ME CAGO TODOS LOS PLANES",NaN,0,2020/03/30,NA,digo lo mismo yo m iba a comprar melo pq en el corte ingles sale a pero la mierda del corona pues me cago todos los planes,"[digo, lo, mismo, yo, m, iba, a, comprar, melo, pq, en, el, corte, ingles, sale, a, pero, la, mierda, del, corona, pues, me, cago, todos, los, planes]","[digo, mismo, m, iba, comprar, melo, pq, corte, ingles, sale, mierda, corona, pues, cago, planes]"
6,1244764670889779201,2020-03-30 23:13:18+00:00,wigmiriam,joder que recuerdos me acaba de dar esto... que nos dijeron que habia una sorpresa y nosotros pensando algo super